# ocean_driven_SST:

SST anomalies caused by anomalous ocean circulation in the eastern equatorial Pacific.

Computes the amount of cooling (warming) by anomalous ocean circulation needed to generate surface temperature anomalies (SSTA) of -1 (1) °C during La Nina (El Nino) events in the eastern equatorial Pacific (horizontal Niño3 average)

#### Reference observations:
TropFlux 1979-2018 (main)

NHF: OAFlux-ISCCP 1984-2009, DEEP-C 1985-2016, ERA5 1940-2022, 20CRv3 1836-2015, NCEP2 1979-2023
SST: ERSSTv5 1854-2023, HadISST 1870-2023, COBE2 1850-2023, ERA5 1940-2022, 20CRv3 1836-2015, NCEP2 1979-2023


In [1]:
from esmvalcore.dataset import Dataset
from esmvalcore.config import CFG
CFG['rootpath']['OBS6'].append(r'/g/data/kj13/datasets/esmvaltool/obsdata-v2') #NCEP-DOE-R2 tos

In [2]:
# varaibles ts, hfds
model_ts_datasets = { 
"ACCESS-ESM1-5": 
    Dataset(
    short_name='tos',
    project='CMIP6',
    mip="Omon",
    exp="historical",
    ensemble="r1i1p1f1",
    timerange="19790101/20190101",
    dataset="ACCESS-ESM1-5",
    grid="gn"
)}
model_ts_datasets["ACCESS-ESM1-5"].add_supplementary(short_name='areacello', mip='Ofx')

model_ts = {name: dataset.load() for name, dataset in model_ts_datasets.items()}

In [3]:
model_hfls_datasets = { 
"ACCESS-ESM1-5": 
    Dataset(
    short_name='hfls',
    project='CMIP6',
    mip="Amon",
    exp="historical",
    ensemble="r1i1p1f1",
    timerange="19790101/20190101",
    dataset="ACCESS-ESM1-5",
    grid="gn"
)}

model_hfls_datasets["ACCESS-ESM1-5"].add_supplementary(short_name='areacella', mip='fx')

#rest variables
model_other = {}
for cvar in ['hfss','rlds','rlus','rsds','rsus']:
    
    model_other[cvar] = model_hfls_datasets["ACCESS-ESM1-5"].copy(short_name=cvar)

# load cubes, formula nhf


In [4]:
model_hfls = {name: dataset.load() for name, dataset in model_hfls_datasets.items()}
model_rest = {name: dataset.load() for name, dataset in model_other.items()}

 hfls: attribute positive not present
loaded from file /g/data/fs38/publications/CMIP6/CMIP/CSIRO/ACCESS-ESM1-5/historical/r1i1p1f1/Amon/hfls/gn/v20191115/hfls_Amon_ACCESS-ESM1-5_historical_r1i1p1f1_gn_185001-201412.nc
 hfls: attribute positive not present
loaded from file 
 hfss: attribute positive not present
loaded from file /g/data/fs38/publications/CMIP6/CMIP/CSIRO/ACCESS-ESM1-5/historical/r1i1p1f1/Amon/hfss/gn/v20191115/hfss_Amon_ACCESS-ESM1-5_historical_r1i1p1f1_gn_185001-201412.nc
 hfss: attribute positive not present
loaded from file 
 rlds: attribute positive not present
loaded from file /g/data/fs38/publications/CMIP6/CMIP/CSIRO/ACCESS-ESM1-5/historical/r1i1p1f1/Amon/rlds/gn/v20191115/rlds_Amon_ACCESS-ESM1-5_historical_r1i1p1f1_gn_185001-201412.nc
 rlds: attribute positive not present
loaded from file 
 rlus: attribute positive not present
loaded from file /g/data/fs38/publications/CMIP6/CMIP/CSIRO/ACCESS-ESM1-5/historical/r1i1p1f1/Amon/rlus/gn/v20191115/rlus_Amon_ACCESS-ESM

In [5]:
# nhf = -*hfls* -hfss + rlds - rlus + rsds - rsus #if 'd' make positive
model_nhf = - model_hfls['ACCESS-ESM1-5'] - model_rest['hfss'] + model_rest['rlds']- model_rest['rlus']+ model_rest['rsds']- model_rest['rsus']

In [6]:
obs_tos_datasets = {
"HadISST": 
    Dataset(
    short_name='tos',
    dataset='HadISST',
    mip="Omon",
    project='OBS',
    type='reanaly',
    timerange="19790101/20190101",
    tier=2),
"TROPFLUX":
    Dataset(
    short_name='tos',
    dataset='TROPFLUX',
    mip="Omon",
    project='OBS6',
    type='reanaly',
    timerange="19790101/20190101",
    tier=2)
}
obs_nhf_datasets = { ##
 # "TROPFLUX": ##netflux data to kj13
 #     Dataset(
 #     short_name='hfds',
 #     dataset='TROPFLUX',
 #     mip="Omon",
 #     project='OBS6',
 #     type='reanaly',
 #     timerange="19790101/20190101",
     # tier=2),
 "ERA-Interim": 
     Dataset(
     short_name='hfds',
     dataset='ERA-Interim',
     mip="Omon",
     project='OBS6',
     type='reanaly',
     timerange="19790101/20190101",
     tier=3)
}

In [7]:
obs_nhf = {name: dataset.load() for name, dataset in obs_nhf_datasets.items()}
obs_tos = {name: dataset.load() for name, dataset in obs_tos_datasets.items()}

In [8]:
from esmvalcore.preprocessor import anomalies
from esmvalcore.preprocessor import area_statistics
from esmvalcore.preprocessor import seasonal_statistics
from esmvalcore.preprocessor import annual_statistics
from esmvalcore.preprocessor import rolling_window_statistics
from esmvalcore.preprocessor import convert_units
from esmvalcore.preprocessor import extract_region
from esmvalcore.preprocessor import extract_season
from esmvalcore.preprocessor import extract_month
from esmvalcore.preprocessor import detrend
from esmvalcore.preprocessor import regrid
from esmvalcore.preprocessor import detrend
from esmvalcore.preprocessor import mask_landsea
from esmvalcore.preprocessor import extract_time
import iris

import matplotlib.pyplot as plt
import iris.quickplot as qplt
import numpy as np
import scipy.stats

In [ ]:
def sst_enso_n34(cube):
    nino34_latext_region = {"start_longitude": 190., "end_longitude": 240., "start_latitude": -5., "end_latitude": 5.}
    cube = convert_units(cube, units="degC")
    # cube = mask_landsea(cube, mask_out="land") #
    cube = extract_region(cube, **nino34_latext_region)
    # #remove time mean
    cube = anomalies(cube, period='monthly') 
    cube = rolling_window_statistics(cube, coordinate='time', operator='mean', window_length=5)##rolling window cuts off months?
    cube = rolling_window_statistics(cube, coordinate='time', operator='mean', window_length=5)
    cube = area_statistics(cube,operator='mean')
    cube = extract_month(cube,12)
    # detrend?
    cube = detrend(cube, dimension="time", method="linear")
    # normalize time series using its temporal standard deviation
    return cube
    
def east(cube):
    nino3_region = {"start_longitude": 210., "end_longitude": 270., "start_latitude": -5., "end_latitude": 5.}
    cube = regrid(cube, target_grid="1x1", scheme="linear")
    # cube = convert_units(cube, units="degC")
    cube = extract_region(cube, **nino3_region)
    # cube = detrend(cube, dimension="time", method="linear")
    cube = rolling_window_statistics(cube, coordinate='time', operator='mean', window_length=5)##rolling window cuts off months?
    cube = rolling_window_statistics(cube, coordinate='time', operator='mean', window_length=5)
    cube = area_statistics(cube, operator='mean')
    cube = anomalies(cube, period="full") #remove seasonal cycle
    
    return cube

In [ ]:
## dsst_nhf
#nhf : season, aunnual
def trend_month_sst(cube_prep): #e.g., dSST[feb] = SST[feb] - SST [jan]
    a_data = cube_prep.data.data
    a1_data = np.append(np.delete(a_data, 0), 0) #add 0 to end
    trend_data = a1_data - a_data #last month value invalid
    cube_prep.data = trend_data
    cube = extract_season(cube_prep,'JASOND')
    cube = annual_statistics(cube, 'sum')[:-1]
    return cube

def dSSTnhf(nhf_east, sst_east):
    dtime = 60 * 60 * 24 * 30.42 #s month-1
    Cp = 4000 #J kg-1 degC-1
    rho = 1024 #kg m-3
    H = 50 #m
    print(nhf_east.shape, sst_east.shape)
    dsstnhf = dtime * nhf_east[:-1] / (sst_east * Cp * rho * H)
    return dsstnhf    

In [10]:
model_nhf_prep = {'ACCESS-ESM1-5': east(model_nhf)}
cube = model_nhf_prep['ACCESS-ESM1-5']
east_cube = extract_season(cube,'JASOND')
print(cube.shape, east_cube.shape)
nhf_east = annual_statistics(east_cube, 'sum')

obs_nhf_prep = {name: east(dataset) for name, dataset in obs_nhf.items()}
cube = obs_nhf_prep['ERA-Interim']
east_cube = extract_season(cube,'JASOND')
print(cube.shape, east_cube.shape)
obs_nhf_east = annual_statistics(east_cube, 'sum')

/g/data/xp65/public/apps/med_conda/envs/esmvaltool-0.6/lib/python3.11/site-packages/iris/cube.py:4881: IrisIgnoringBoundsWarning: The bounds of coordinate 'time' were ignored in the rolling window operation.
  warnings.warn(
/g/data/xp65/public/apps/med_conda/envs/esmvaltool-0.6/lib/python3.11/site-packages/iris/cube.py:4881: IrisIgnoringBoundsWarning: The bounds of coordinate 'time' were ignored in the rolling window operation.
  warnings.warn(
/g/data/xp65/public/apps/med_conda/envs/esmvaltool-0.6/lib/python3.11/site-packages/iris/coords.py:2182: IrisVagueMetadataWarning: Collapsing a non-contiguous coordinate. Metadata may not be fully descriptive for 'time'.
  warnings.warn(
/g/data/xp65/public/apps/med_conda/envs/esmvaltool-0.6/lib/python3.11/site-packages/iris/cube.py:4881: IrisIgnoringBoundsWarning: The bounds of coordinate 'time' were ignored in the rolling window operation.
  warnings.warn(
/g/data/xp65/public/apps/med_conda/envs/esmvaltool-0.6/lib/python3.11/site-packages/iri

(424,) (212,)
(472,) (236,)


In [21]:

model_ts_prep = {name: east(dataset) for name, dataset in model_ts.items()}
obs_ts_prep = {name: east(dataset) for name, dataset in obs_tos.items()}

obs_sst = trend_month(obs_ts_prep['TROPFLUX'])
mod_sst = trend_month(model_ts_prep['ACCESS-ESM1-5'])

# qplt.plot(obs_sst)
# qplt.plot(mod_sst, label='model')

model = dSSTnhf(nhf_east, mod_sst)
obs = dSSTnhf(obs_nhf_east, obs_sst)

/g/data/xp65/public/apps/med_conda/envs/esmvaltool-0.6/lib/python3.11/site-packages/iris/cube.py:4881: IrisIgnoringBoundsWarning: The bounds of coordinate 'time' were ignored in the rolling window operation.
  warnings.warn(
/g/data/xp65/public/apps/med_conda/envs/esmvaltool-0.6/lib/python3.11/site-packages/iris/coords.py:2182: IrisVagueMetadataWarning: Collapsing a non-contiguous coordinate. Metadata may not be fully descriptive for 'time'.
  warnings.warn(
/g/data/xp65/public/apps/med_conda/envs/esmvaltool-0.6/lib/python3.11/site-packages/iris/cube.py:4881: IrisIgnoringBoundsWarning: The bounds of coordinate 'time' were ignored in the rolling window operation.
  warnings.warn(
/g/data/xp65/public/apps/med_conda/envs/esmvaltool-0.6/lib/python3.11/site-packages/iris/cube.py:4881: IrisIgnoringBoundsWarning: The bounds of coordinate 'time' were ignored in the rolling window operation.
  warnings.warn(
/g/data/xp65/public/apps/med_conda/envs/esmvaltool-0.6/lib/python3.11/site-packages/iri

In [24]:
model_ts_enso = {name: sst_enso_n34(dataset) for name, dataset in model_ts.items()}
obs_tos_enso = {name: sst_enso_n34(dataset) for name, dataset in obs_tos.items()}

/g/data/xp65/public/apps/med_conda/envs/esmvaltool-0.6/lib/python3.11/site-packages/iris/cube.py:4881: IrisIgnoringBoundsWarning: The bounds of coordinate 'time' were ignored in the rolling window operation.
  warnings.warn(
/g/data/xp65/public/apps/med_conda/envs/esmvaltool-0.6/lib/python3.11/site-packages/iris/cube.py:4881: IrisIgnoringBoundsWarning: The bounds of coordinate 'time' were ignored in the rolling window operation.
  warnings.warn(
/g/data/xp65/public/apps/med_conda/envs/esmvaltool-0.6/lib/python3.11/site-packages/iris/coords.py:2171: IrisVagueMetadataWarning: Cannot check if coordinate is contiguous: Invalid operation for 'cell index along second dimension', with 0 bound(s). Contiguous bounds are only defined for 1D coordinates with 2 bounds. Metadata may not be fully descriptive for 'cell index along second dimension'. Ignoring bounds.
  warnings.warn(
/g/data/xp65/public/apps/med_conda/envs/esmvaltool-0.6/lib/python3.11/site-packages/iris/coords.py:2171: IrisVagueMetad

In [25]:
# detect nino and nina
from esmvalcore.preprocessor import mask_above_threshold, mask_below_threshold
def enso_events(cube):
    a_events = mask_to_years(mask_above_threshold(cube.copy(), -0.75))
    o_events = mask_to_years(mask_below_threshold(cube.copy(), 0.75))
    return {'nina':a_events, 'nino':o_events}

def mask_to_years(events):
    maskedTime = np.ma.masked_array(events.coord('time').points, mask=events.data.mask)
    return [events.coord('time').units.num2date(time).year for time in maskedTime.compressed()]

def dSSTnhf_ninonina(events_dict, dsstnhf):
    datapoints = []
    for enso, events in events_dict.items():
        year_enso = iris.Constraint(time=lambda cell: cell.point.year in events)
        cube = dsstnhf.extract(year_enso)
        ## regroup and average
        datapoints.append(cube.data)
        print(enso, [d for d in cube.data.data])
    return datapoints

In [26]:
events = enso_events(model_ts_enso['ACCESS-ESM1-5'])
print(events)
events = enso_events(obs_tos_enso['TROPFLUX'])
print(events)

# testdata = dSSTnhf_ninonina(events, dsstnhf)

{'nina': [1984, 1985, 1990, 1993, 1999, 2006, 2011], 'nino': [1979, 1983, 1995, 1998, 2001, 2013]}
{'nina': [1983, 1984, 1988, 1995, 1998, 1999, 2007, 2010, 2011], 'nino': [1982, 1986, 1987, 1991, 1994, 1997, 2002, 2009, 2015]}


In [76]:
lsall =[]
for mem in testdata:
    mean = np.mean(mem)
    ls = [d for d in mem]
    lsall += ls
    print(mean)

1 - np.mean(lsall) 
#wiki : dSSToce = dSST - dSSTnhf, spreadsheet 1-averaged data

1.2725377823773388e-07
-2.5762508422937425e-06


-1.1205175850844859e-06

# Diagnostic Level 1

In [ ]:
# scatter plot metric


# metric_val = abs((mod_slope-slope)/slope)*100

print(model_nhf_prep["ACCESS-ESM1-5"].shape, model_ts_prep["ACCESS-ESM1-5"].shape) #scatter plot
print(model_nhf_prep["ACCESS-ESM1-5"].units, model_ts_prep["ACCESS-ESM1-5"].units)
plt.scatter(model_ts_prep["ACCESS-ESM1-5"].data, model_nhf_prep["ACCESS-ESM1-5"].data, s=10)


# print(obs_tos_prep["HadISST"].shape, obs_tauu_prep["ERA-Interim"].shape)
# plt.scatter(obs_tos_prep["HadISST"].data, obs_tauu_prep["ERA-Interim"].data,s=20, c='black', marker='D')

plt.ylabel(f'dSSToce (degC)')

## text slope
plt.text(0.02, 0.9, f'metric_val ', fontsize=12, ha='left',
         transform=plt.gca().transAxes, bbox=dict(facecolor='white', alpha=0.8, edgecolor='none'))

# print(metric_val)

# Diagnostic Level 2

In [ ]:
# SPLIT BY X<0 AND X>0 (temp)
import pandas as pd

model_tauu_prep["ACCESS-ESM1-5"].data, model_ssh_prep["ACCESS-ESM1-5"]
# matrix/cube? split by tauua 0 -df?time rows, col tauu,ssh create tuple points, zip

df = pd.DataFrame({'tauu':model_tauu_prep["ACCESS-ESM1-5"].data, 'ssh':model_ssh_prep["ACCESS-ESM1-5"].data})
print(df.shape)
df.loc[df['tauu']<0]

xseq = np.linspace(-50, 0, num=50)
slope, intcpt, _,_,_ = linregress(df.loc[df['tauu']<0, 'tauu'], df.loc[df['tauu']<0, 'ssh'])
plt.plot(xseq, intcpt+slope*xseq, linewidth=3)
print(slope)
xseq = np.linspace(0, 50, num=50)
slope, intcpt, _,_,_ = linregress(df.loc[df['tauu']>0, 'tauu'], df.loc[df['tauu']>0, 'ssh'])
plt.plot(xseq, intcpt+slope*xseq, color='red', linewidth=3)
print(slope)

plt.scatter(model_tauu_prep["ACCESS-ESM1-5"].data, model_ssh_prep["ACCESS-ESM1-5"].data, c='k', s=10)
xseq = np.linspace(-50, 50, num=50)
slope, intcpt, _,_,_ = linregress(df['tauu'], df['ssh'])
plt.plot(xseq, intcpt+slope*xseq, c='black')
print(slope)
plt.xlim(-50,50)
plt.ylim(-25,25)
plt.grid(linestyle='--')